In [28]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer
import numpy as np

In [29]:
# !pip install pyarrow
# !pip install fastparquet

In [30]:
# Q1
df = pd.read_parquet('yellow_tripdata_2023-01.parquet')
print(df.shape[1])

19


In [31]:
df.dtypes # already datetimes
# otherwise, we would have needed:
# df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
# df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# the duration in minutes
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [32]:
df.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [33]:
# Q2
df['duration'].std()

42.59435124195458

In [34]:
# Q3
# Filter the DataFrame to keep only records with duration between 1 and 60 minutes
filtered_df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Calculate the fraction of records left
fraction_left = len(filtered_df) / len(df)

print(fraction_left)

0.9812202822125979


In [35]:
# Q4
# Select the relevant columns and convert IDs to strings
filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)

# Turn the DataFrame into a list of dictionaries
data_dicts = filtered_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Fit a dictionary vectorizer
dv = DictVectorizer()
feature_matrix = dv.fit_transform(data_dicts)

print(feature_matrix.shape)

C:\Users\Utilisteur\AppData\Local\Temp\ipykernel_45216\1682795159.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
C:\Users\Utilisteur\AppData\Local\Temp\ipykernel_45216\1682795159.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)


(3009173, 515)


In [36]:
# Q5
# Assuming 'duration' is the target variable
X = feature_matrix
y = filtered_df['duration'].values

# Train the linear regression model
model = LinearRegression()
model.fit(X, y)

# Make predictions on the training data
y_pred = model.predict(X)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))

print(f'RMSE: {rmse}')

RMSE: 7.649261936284003


In [37]:
# Q6
# Load the validation dataset
df_val = pd.read_parquet('yellow_tripdata_2023-02.parquet')

# Convert the datetime columns
df_val['tpep_pickup_datetime'] = pd.to_datetime(df_val['tpep_pickup_datetime'])
df_val['tpep_dropoff_datetime'] = pd.to_datetime(df_val['tpep_dropoff_datetime'])

# Calculate the duration in minutes
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter the DataFrame to keep only records with duration between 1 and 60 minutes
df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)]

# Convert IDs to strings
df_val['PULocationID'] = df_val['PULocationID'].astype(str)
df_val['DOLocationID'] = df_val['DOLocationID'].astype(str)

# Turn the DataFrame into a list of dictionaries
data_dicts_val = df_val[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Transform the validation data using the fitted DictVectorizer
X_val = dv.transform(data_dicts_val)

# Extract the target variable
y_val = df_val['duration'].values

# Make predictions on the validation data
y_pred_val = model.predict(X_val)

# Calculate the RMSE
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))

# Print the RMSE
print(f'RMSE on validation data: {rmse_val}')

RMSE on validation data: 7.811818654341152
